In [48]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import VGG19
import tensorflow as tf
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# Data

Our data will is in the DATASET folder split into our TRAIN and TEST datasets. Each folder contains one folder for each of our labels. Our next step will be to preprocess our data. 

* We use ImageDataGenerator to do some transformations of the images
* Then we can apply the transformations to the directories for our train and test folder

In [49]:
train_dir = 'DATASET/TRAIN'
test_dir = 'DATASET/TEST'

In [70]:
train_datagen = ImageDataGenerator(width_shift_range = 0.1,
                                  horizontal_flip = True,
                                  rescale = 1./255,
                                  validation_split = 0.2,
                                  preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(rescale =1./255,
                                 validation_split = 0.2,
                                 preprocessing_function=preprocess_input)

In [71]:
train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                   target_size = (224,224),
                                                   color_mode = 'rgb',
                                                   class_mode = 'categorical',
                                                    batch_size = 16,
                                                   subset = 'training')
validation_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                       target_size = (224,224),
                                                       color_mode = 'rgb',
                                                       class_mode = 'categorical',
                                                       batch_size = 16,
                                                       subset = 'validation')

Found 866 images belonging to 5 classes.
Found 92 images belonging to 5 classes.


In [72]:
print(validation_generator.class_indices)
print(train_generator.class_indices)

{'downdog': 0, 'goddess': 1, 'plank': 2, 'tree': 3, 'warrior2': 4}
{'downdog': 0, 'goddess': 1, 'plank': 2, 'tree': 3, 'warrior2': 4}


We will use the VGG19 model you can read more about the requirements and considerations for this model in the documentation (https://keras.io/api/applications/vgg/).


In [73]:
model = VGG19(include_top = False,weights = 'imagenet',input_shape= (224,224,3))

# Freeze the imported layers so they cannot be retrained.
for layer in model.layers:
    layer.trainable = False
    

model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

### Adding flattening and dense layers

Right now, our model is missing a top to actually classify our features. Let's add them:

In [74]:
from keras import Sequential
from keras.layers import Dense
from keras.layers import Flatten

new_model = Sequential()
new_model.add(model)
new_model.add(Flatten())
new_model.add(Dense(5,activation = 'softmax'))

# Summarize.
new_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 7, 7, 512)         20024384  
_________________________________________________________________
flatten_2 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 125445    
Total params: 20,149,829
Trainable params: 125,445
Non-trainable params: 20,024,384
_________________________________________________________________


In [77]:
from tensorflow.keras.optimizers import Adam
# Compile and fit the model. Use the Adam optimizer and crossentropical loss. 
# Use the validation data argument during fitting to include your validation data.
optimizer = Adam(learning_rate = 0.0001)
new_model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
history = new_model.fit(train_generator,
                        epochs=1, 
                        batch_size=16,
                        validation_data=validation_generator
                       )

55/55 [==============================] - 305s 6s/step - loss: 0.1116 - accuracy: 0.9815 - val_loss: 1.6311 - val_accuracy: 0.8370


# Predicting the class of your image

Let's take this bad boy for a spin! Can your image get properly identified?

In [85]:
import numpy as np
from tensorflow.keras.applications.vgg19 import preprocess_input

# Predict the class of your picture.

img = tf.keras.preprocessing.image.load_img("./test_folder/downward_dog_new.jpg", target_size = (224, 224))


img_nparray = tf.keras.preprocessing.image.img_to_array(img)

print(img_nparray.shape)
#convert image to array

x = preprocess_input(img_nparray).reshape((1,224,224,3))

print(x.shape)

prediction = new_model.predict(x)

print(prediction.shape)

# create a list containing the class labels
class_labels = ['downdog', 'goddess', 'plank', 'tree', 'warrior2']

# find the index of the class with maximum score
pred = np.argmax(prediction, axis=-1)
class_labels[pred[0]]

(224, 224, 3)
(1, 224, 224, 3)
(1, 5)


'downdog'